In [1]:
import numpy as np
import scipy.io
import shutil
import os
from scipy.io.wavfile import write
from scipy.signal import resample
import csv

from parameters_CINC11CINC17_unbalanced import parameters
# from parameters_CINC11CINC17_unbalanced import parameters
from data_storage_utils import create_directory, normalizer, generate_store_kramer_data

parameters = parameters()

In [2]:
import matplotlib.pyplot as plt

In [3]:
path_training_data = parameters['annotation_file_train'] # location of anntotation file
path_testing_data =  parameters['annotation_file_test'] # location of anntotation file
path_cinc11 = parameters['path_cinc11']
path_cinc17 = parameters['path_cinc17']
path_save = parameters['path_training_data']

In [5]:
data_length = parameters['data_length']
data_length_cinc17_300hz = parameters['data_length_cinc17_300hz']
wav_data_range = parameters['wav_data_range']
amount_data = parameters['amount_data'] # number of datapoints we want per label

In [6]:
label_count = [0, 0] # nubmer of current datapoints per label

In [7]:
# only for debugging, remove existing dataset by removing folder:
# shutil.rmtree(path_save+"\\0")
# shutil.rmtree(path_save+"\\1")

In [8]:
# Generate the folders for storing the data

# def create_directory(output_path):
#     """
#     output_path: the path from source to the output folder 
    
#     creates the folders needed to store the output data
#     """
#     if not os.path.exists(output_path):
#         os.makedirs(output_path)
#     if not os.path.exists(output_path+'/0'):
#         os.makedirs(output_path+'/0')
#     if not os.path.exists(output_path+'/1'):
#         os.makedirs(output_path+'/1')
#     if not os.path.exists(output_path+'/rest'):
#         os.makedirs(output_path+'/rest')
        
#         print('output directories created')

# create_directory(path_save)

Normalization function used:

$data_{normalized}\mapsto \frac{data-r_{\text{min}}}{r_{\text{max}}-r_{\text{min}}}\times (t_{\text{max}}-t_{\text{min}}) + t_{\text{min}}$

Get the annotation of the data and save it in the `annotations` array and their corresponding recording numbers and save it in the `recording numbers` array. The `generate_store_kramer_data` function finds the location where the data is stored. The name of the data file is the recording number. The data is then saved as a `.wav` file in a folder named according to the annotation number. E.G. : `\\Code\\kramer_data\\0\\1002603_10.wav`

In [9]:
annotations = []

with open(path_training_data+"\\Annotation Training Set Group I.txt") as f:
    lines = f.readlines()
    for line in lines: 
        line = line.split('\t')
        annotations_12 = []
        for val in line: 
            annotations_12.append(val[0])
        annotations.append(annotations_12)
        
recording_numbers = []

with open(path_training_data+"\\Training 50 Group I recording names.txt") as f:
    line = f.readline() 
    line = line.split('\t')
    for val in line: 
        val = val.split('\n')
        recording_numbers.append(val[0])
        
label_count = generate_store_kramer_data(annotations, recording_numbers, wav_data_range, label_count, path_cinc11, path_save, amount_data)

In [11]:
annotations = []

with open(path_training_data+"\\Annotation Training Set Group II.txt") as f:
    lines = f.readlines()
    for line in lines: 
        line = line.split('\t')
        annotations_12 = []
        for val in line: 
            annotations_12.append(val[0])
        annotations.append(annotations_12)
        
recording_numbers = []

with open(path_training_data+"\\Training 50 Group II recording names.txt") as f:
    line = f.readline() 
    line = line.split('\t')
    for val in line: 
        val = val.split('\n')
        recording_numbers.append(val[0])
        
label_count = generate_store_kramer_data(annotations, recording_numbers, wav_data_range, label_count, path_cinc11, path_save, amount_data)

In [13]:
annotations = []

with open(path_testing_data+"\\Annotation Testing Set Group I.txt") as f:
    lines = f.readlines()
    for line in lines: 
        line = line.split('\t')
        annotations_12 = []
        for val in line: 
            annotations_12.append(val[0])
        annotations.append(annotations_12)  

recording_numbers = []

with open(path_testing_data+"\\Testing 175 Group I recording names.txt") as f:
    lines = f.readlines() 
    for val in lines:
        val = val.split('\n')
        recording_numbers.append(val[0])

# print(recording_numbers)
label_count = generate_store_kramer_data(annotations, recording_numbers, wav_data_range, label_count, path_cinc11, path_save, amount_data)

In [14]:
annotations = []

with open(path_testing_data+"\\Annotation Testing Set Group II.txt") as f:
    lines = f.readlines()
    for line in lines: 
        line = line.split('\t')
        annotations_12 = []
        for val in line: 
            annotations_12.append(val[0])
        annotations.append(annotations_12)
        
recording_numbers = []

with open(path_testing_data+"\\Testing 175 Group II recording names.txt") as f:
    lines = f.readlines() 
    for val in lines:
        val = val.split('\n')
        recording_numbers.append(val[0])
        
label_count = generate_store_kramer_data(annotations, recording_numbers, wav_data_range, label_count, path_cinc11, path_save, amount_data)

In [18]:
# save labels 
commands = [0, 1]
commands_count = [0, 0]

dict_labels = {}
with open(path_cinc17+"\\REFERENCE.csv", mode='r') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[1] == '~':
            dict_labels[f'{row[0]}'] = commands[0]
            commands_count[0] += 1
        else: 
            dict_labels[f'{row[0]}'] = commands[1]
            commands_count[1] += 1
print('Commands:', commands)
print('Amount CinC11 data per label:', commands_count)

Commands: [0, 1]
Amount CinC11 data per label: [284, 8244]


Top up the Kramer data with CINC17 data to get enough datapoints.

In [19]:
with open(path_cinc17+"\\training2017\\RECORDS") as f:
    lines = f.readlines()
    file_names = []        

    for i, record_nr in enumerate(lines): 
        data_path = path_cinc17+"\\"f"{record_nr[0:6]}.mat"
        file_names.append(data_path)
        
        label = dict_labels[f'{record_nr[0:6]}']

        data = scipy.io.loadmat(data_path)
        data = data.get('val').flatten()
        data = normalizer(data, wav_data_range)

        if len(data) < data_length_cinc17_300hz:
            pass
        else: 
            if label_count[label] < amount_data:
                if label == 0: # if noisy data 
                    data = data[:data_length_cinc17_300hz]                     
                    resampled_data = resample(data, data_length)
                    resampled_data = np.asarray(resampled_data, dtype=np.int16)
                    write(path_save+"\\"f"{label}\\{record_nr[0:6]}.wav", 500, resampled_data)
                    label_count[label] += 1
                    
                else: # if not noisy data 
                    data = data[:data_length_cinc17_300hz]
                    resampled_data = resample(data, data_length)
                    resampled_data = np.asarray(resampled_data, dtype=np.int16)
                    write(path_save+"\\"f"{label}\\{record_nr[0:6]}.wav", 500, resampled_data)
                    label_count[label] += 1

In [20]:
print(label_count)

[2024, 11888]


In [21]:
# shutil.rmtree(path_save+"\\rest")